# IMPORTS

In [8]:
# import pandas as pd

import my_pickle as mp
import my_split as ms
import my_dataframes as md

# from importlib import reload

# import warnings
# warnings.filterwarnings('ignore')

In [ ]:
# reload(md)
# reload(ms)

# READ IN

In [3]:
message_df = md.get_message_df()
convo_df = md.get_conversation_df(message_df)
user_df = md.get_user_df()

created message dataframe
created conversation dataframe
... read in dataframe
... dropped users that aren't onboarded
... renamed columns
... changed to datetimes
... dropped columns
... got neighborhoods
... got metro areas
... added new features
... converted to binary
Created user dataframe


# ECT SPLIT

In [10]:
# find the split!
cutoff = ms.ect_find_split(convo_df, .7)
print("split on:                      {}".format(cutoff))

# do the split!
safe_convo_df = ms.ect_make_split(convo_df, cutoff)
print("most recent convo data:        {}".format(safe_convo_df.timestamp.max()))

safe_message_df = ms.ect_make_split(message_df, cutoff)
print("most recent message data:      {}".format(safe_message_df.timestamp.max()))

split on:                      2017-10-01 06:37:20.466000
most recent convo data:        2017-10-01 06:37:20.466000
most recent message data:      2017-10-01 06:37:20.466000


# PICKLE EVERYTHING!

https://www.youtube.com/watch?v=yYey8ntlK_E

In [11]:
mp.json_it(user_df,'data_user')
mp.json_it(safe_message_df,'data_message')  
mp.json_it(safe_convo_df,'data_convo')  